In [1]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 构建分类器
class SurnameClassifier(nn.Module):
    # 初始化，构建各层模型初值
    def __init__(self, initial_num_channels, num_classes, num_channels):
        super(SurnameClassifier, self).__init__()
        self.convent = nn.Sequential(
            nn.Conv1d(in_channels = initial_num_channels,
                     out_channels = num_channels, kernel_size = 3),
            nn.ELU(),
            nn.Conv1d(in_channels = num_channels, out_channels = num_channels,
                     kernel_size = 3, stride = 2),
            nn.ELU(),
            nn.Conv1d(in_channels = num_channels, out_channels = num_channels,
                     kernel_size = 3),
            nn.ELU()
        )
        self.fc = nn.linear(num_channels, num_classes)
    # 前向传播
    def forward(self, x_surname, apply_softmax = False):
        # 默认 不调用softmax
        features = self.convent(x_surname).squeeze(dim=2)
        prediction_vector = self.fc(features)
        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim = 1)
        return prediction_vector

In [ ]:
# 原始数据 预处理
class SurnnameDataSet(Dataset):
    def __getitem__(self, index):
        row = self.target_df.iloc[index]
        surname_matrix = 
            self.vectorizer.vectorize(row.surname, self._max_seq_length)
        nationality_index = 
            self.vectorizer.nationality_vocab.lookup_token(row.nationality)
        return {'x_surname': surname_matrix,
                'y_natinoality': nationality_index}


In [ ]:
# 数据 向量化
class SurnameVectorizer(object):
    def vectorize(self, surname):
        one_hot_matrix_size = (len(self.character_vocab), self.max_surname_length)
        one_hot_martix = np.zeros(one_hot_matrix_size, dtype = np.float32)
        
        for position_index, character in enumerate(surname):
            character_index = self.character_vocab.lookup_token(character)
            one_hot_matrix[character_index][position_index] = 1
        return one_hot_matrix
    def from_dataframe(cls, surname_df):
        character_vocab = Vocabulary(unk_token="@")
        nationality_vocab = Vocabulary(add_unk = False)
        max_surname_length = 0
        for index, row in surname_df.iterrows():
            max_surname_length = max(max_surname_length, len(row.surname))
            for letter in row.surname:
                character_vocab.add_token(letter)
            nationality_vocab.add_token(row.nationality)
        
        return cls(character_vocab, nationality_vacab, max_surname_length)

In [ ]:
# 模型设置
args = Namespace(
    surname_csv = "surnames_with_splits.csv",
    vectorizer_file = "vectorizer.json",
    model_state_file = "model.pth",
    save_dir = "model_storage/ch4/cnn",
    hidden_dim = 100,
    num_channels = 256,
    seed = 1337,
    learning_rate = 0.001,
    batch_size = 128,
    num_epochs = 100,
    early_stopping_criteria = 5,
    dropout_p = 0.1
)

In [ ]:
# 预测
def predict_nationality(surname, classifier, vectorizer):
    vectorized_surname = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(0)
    result = classifier(vectorized_surname, apply_softmax = True)
    
    probability_values, indices = result.max(dim = 1)
    index = indices.item()
    
    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)
    probability_value = probability_values.item()
    return {'nationality': predicted_nationalitym
           'probability': probability_value}